In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

import data

convs = list(data.convs())

In [45]:
df

utter         act  \
0     person_a  Say , Jim , how about going for a few beers af...   directive   
      person_b   You know that is tempting but is really not g...  commissive   
      person_a     What do you mean ? It will help us to relax .     question   
      person_b   Do you really think so ? I don't . It will ju...    question   
      person_a   I guess you are right.But what shall we do ? ...    question   
      person_b   I suggest a walk over to the gym where we can...   directive   
      person_a   That's a good idea . I hear Mary and Sally of...  commissive   
      person_b   Sounds great to me ! If they are willing , we...      inform   
      person_a                             Good.Let ' s go now .    directive   
      person_b                                       All right .   commissive   
1     person_a                             Can you do push-ups ?     question   
      person_b   Of course I can . It's a piece of cake ! Beli...      inform   
      person_a              Really ? I think that's impossible !     question   
      person_b                            You mean 30 push-ups ?     question   
      person_a                                            Yeah !       inform   
      person_b   It's easy . If you do exercise everyday , you...      inform   
2     person_a                 Can you study with the radio on ?     question   
      person_b               No , I listen to background music .       inform   
      person_a                          What is the difference ?     question   
      person_b                The radio has too many comerials .       inform   
      person_a   That's true , but then you have to buy a reco...      inform   
3     person_a                               Are you all right ?     question   
      person_b   I will be all right soon . I was terrified wh...      inform   
      person_a                    Don't worry.He is an acrobat 。       inform   
      person_b                                           I see .       inform   
4     person_a           Hey John , nice skates . Are they new ?     question   
      person_b   Yeah , I just got them . I started playing ic...      inform   
      person_a                       What position do you play ?     question   
      person_b   I ’ m a defender . It ’ s a lot of fun . You ...      inform   
      person_a   Yeah , you ’ re a pretty big guy . I play goa...      inform   
...                                                           ...         ...   
11113 person_b   Then would you kindly come to our shop with t...  commissive   
      person_a                               Thank you so much !       inform   
11114 person_a                 Do you have any seats available ?     question   
      person_b                           Yes . This way please .       inform   
      person_a                        What's the special today ?     question   
      person_b               I recommend our New English Pizza .       inform   
      person_a   Then give me a large one and an egg drop soup .    directive   
      person_b                     Okay . Wait a minute please .   commissive   
11115 person_a  Uncle Ben , how did the Forbidden City get thi...    question   
      person_b   Well , in the feudal society , emperors had s...      inform   
      person_a       His residence ? You mean the whole palace ?     question   
      person_b   Yes . The Forbidden City is divided into two ...      inform   
      person_a      Oh , unbelievable . This was too luxurious .       inform   
      person_b   Even the whole country belonged to him . That...      inform   
      person_a   OK , then . I love these unique structured to...      inform   
      person_b   Yeah , the buildings and designs of the Forbi...      inform   
      person_a   I agree . It's definitely a product of wisdom...    question   
      person_b   Some of them are mascots and some are symbols...      inform   
      pers